In [1]:
import nltk
import numpy as py
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

import re
import numpy as np
import pandas as pd
from pprint import pprint

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from collections import Counter
from gensim.test.utils import datapath
import pickle

In [9]:
def retrieve_feature_label(path):
    df = pd.read_pickle(path)
    return df['total_notes'], df['label']

def clear(x):
    stop_words = set(stopwords.words('english'))
    x=str(x)
    x=x.lower()
    word_tokens=nltk.word_tokenize(x)
    word_tokens=[lemmatizer.lemmatize(x) for x in word_tokens]
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return filtered_sentence

def check_token(X):
	all_token=[]
	for x in X:
		all_token += x
		print(Counter(all_token).most_common(100))

def update_stop_words(addlist):
	stop_words = set(stopwords.words('english'))
	for x in addlist:
		stop_words.add(x)
	return stop_words

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts, stop_words):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts, trigram_mod, bigram_mod ):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(nlp, texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


def process_X(X):
 	addlist=['.',',','cw','spoke','!','?']
 	stop_words = update_stop_words(addlist)
 	data = X.values.tolist()
 	data_words = list(sent_to_words(data))
 	# Build the bigram and trigram models
 	bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
 	trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
 	# Faster way to get a sentence clubbed as a trigram/bigram
 	bigram_mod = gensim.models.phrases.Phraser(bigram)
 	trigram_mod = gensim.models.phrases.Phraser(trigram)
 	print("Finish bigram")
 	print("-------------------------------------------")
 	# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
 	nlp = spacy.load('en',disable=['parser', 'ner']) 	# python3 -m spacy download en
 	data_words_nostops = remove_stopwords(data_words, stop_words) 	# Remove Stop Words
 	data_words_bigrams = make_bigrams(data_words_nostops, bigram_mod)	 	# Form Bigrams
 	data_lemmatized = lemmatization(nlp, data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']) 	# Do lemmatization keeping only noun, adj, vb, adv
 	id2word = corpora.Dictionary(data_lemmatized)
 	# Create Corpus
 	texts = data_lemmatized
 	# Term Document Frequency
 	corpus = [id2word.doc2bow(text) for text in texts]
 	print("Finish building corpus")
 	print("-------------------------------------------")
 	lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=6, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

 	return corpus, lda_model



def save_model(corpus, model):
	model.save('../../Youth_Recidivism_Project/LDA_Output/lda.model')
	with open("../../Youth_Recidivism_Project/LDA_Output/corpus.txt", "wb") as p:   #Pickling
		pickle.dump(corpus, p)


In [10]:
X, Y = retrieve_feature_label("../../Youth_Recidivism_Project/Dataset/train.pickle")
X = X.apply(clear)
corpus, model = process_X(X)
save_model(corpus, model)

Finish bigram
-------------------------------------------
Finish building corpus
-------------------------------------------


In [11]:
lda_model = gensim.models.ldamodel.LdaModel.load('../../Youth_Recidivism_Project/LDA_Output/lda.model')
print(lda_model.print_topics())
with open("../../Youth_Recidivism_Project/LDA_Output/corpus.txt", "rb") as fp:   # Unpickling
    corpus = pickle.load(fp)
predicted_labels = lda_model.get_document_topics(corpus)
new_label = []
prob = []
for i in predicted_labels:
    label,probability = max(i, key=lambda x:x[1])
    new_label.append(label)
    prob.append(probability)
new_label = np.array(new_label)

[(0, '0.059*"also" + 0.042*"client" + 0.040*"discuss" + 0.035*"youth" + 0.033*"family" + 0.026*"plan" + 0.023*"sign" + 0.021*"need" + 0.020*"job" + 0.018*"community"'), (1, '0.101*"call" + 0.051*"writer" + 0.051*"check" + 0.047*"know" + 0.041*"schedule" + 0.040*"phone" + 0.034*"tomorrow" + 0.034*"receive" + 0.033*"let" + 0.029*"email"'), (2, '0.074*"go" + 0.047*"home" + 0.046*"would" + 0.035*"mother" + 0.033*"state" + 0.028*"ask" + 0.027*"say" + 0.027*"tell" + 0.026*"get" + 0.025*"inform"'), (3, '0.079*"program" + 0.031*"make" + 0.027*"treatment" + 0.019*"staff" + 0.019*"present" + 0.017*"regard" + 0.017*"concern" + 0.014*"assessment" + 0.014*"process" + 0.013*"behavior"'), (4, '0.097*"case" + 0.090*"mom" + 0.067*"transport" + 0.065*"court" + 0.061*"date" + 0.050*"update" + 0.035*"agree" + 0.033*"open" + 0.032*"release" + 0.026*"informed"'), (5, '0.069*"report" + 0.064*"meet" + 0.054*"work" + 0.050*"today" + 0.049*"school" + 0.045*"time" + 0.041*"well" + 0.037*"meeting" + 0.032*"visit"

In [12]:
z = [0] * len(new_label)
pd_new = pd.DataFrame({'new':new_label,"z":z})
pd_new.columns = ['new','num']
pd_new_stat = pd_new.groupby("new").count()
print(pd_new_stat)

       num
new       
0    11398
1     3754
2    78909
3    12833
4      471
5    38952


In [13]:
print(max(new_label))

5


In [14]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
pyLDAvis.save_html(vis, '../../Youth_Recidivism_Project/LDA_Output/lda.html')